# Vectorisation de Texte: OneHotEncoder et Bag of Words

**Suite du prétraitement NLP - Données de main.ipynb**

## 0. Chargement des données (à exécuter d'abord!)

Charger les données prétraitées depuis main.ipynb

In [2]:
import subprocess
import sys
import pickle
import os

print("=" * 70)
print("CHARGEMENT DES DONNÉES DE main.ipynb")
print("=" * 70 + "\n")

# Vérifier si les variables existent déjà (si main.ipynb a été exécuté avant)
try:
    # Tester si cleaned_tokens existe
    test = cleaned_tokens
    print("✓ Variables trouvées dans le kernel actuel")
    print(f"  • cleaned_tokens: {len(cleaned_tokens)} mots")
    data_loaded = True
except NameError:
    data_loaded = False
    print("⚠️  cleaned_tokens non disponible dans le kernel actuel\n")
    
    # Essayer de charger depuis un fichier pickle
    pickle_file = 'cleaned_tokens.pkl'
    if os.path.exists(pickle_file):
        print(f"✓ Fichier {pickle_file} trouvé!")
        try:
            with open(pickle_file, 'rb') as f:
                cleaned_tokens = pickle.load(f)
            print(f"✓ Données chargées: {len(cleaned_tokens)} tokens")
            data_loaded = True
        except Exception as e:
            print(f"✗ Erreur lors du chargement: {e}")
    else:
        print(f"✗ Fichier {pickle_file} non trouvé\n")
        print("SOLUTIONS DE CHARGEMENT:")
        print("─" * 70)
        print("\n1️⃣  MÉTHODE RECOMMANDÉE (Même kernel):")
        print("   ✓ Ouvrir main.ipynb dans le même VS Code")
        print("   ✓ Exécuter TOUTES les cellules (Ctrl+Alt+Entrée)")
        print("   ✓ Revenir à main2.ipynb et exécuter cette cellule\n")
        
        print("2️⃣  MÉTHODE ALTERNATIVE (Sauvegarder/Charger):")
        print("   ✓ Dans main.ipynb, ajouter à la fin:")
        print("     import pickle")
        print("     with open('cleaned_tokens.pkl', 'wb') as f:")
        print("         pickle.dump(cleaned_tokens, f)")
        print("   ✓ Exécuter la cellule")
        print("   ✓ Revenir à main2.ipynb\n")

if data_loaded:
    print("\n" + "=" * 70)
    print("✓ DONNÉES CHARGÉES AVEC SUCCÈS")
    print("=" * 70)
    print(f"Prêt pour la vectorisation!\n")

CHARGEMENT DES DONNÉES DE main.ipynb

⚠️  cleaned_tokens non disponible dans le kernel actuel

✗ Fichier cleaned_tokens.pkl non trouvé

SOLUTIONS DE CHARGEMENT:
──────────────────────────────────────────────────────────────────────

1️⃣  MÉTHODE RECOMMANDÉE (Même kernel):
   ✓ Ouvrir main.ipynb dans le même VS Code
   ✓ Exécuter TOUTES les cellules (Ctrl+Alt+Entrée)
   ✓ Revenir à main2.ipynb et exécuter cette cellule

2️⃣  MÉTHODE ALTERNATIVE (Sauvegarder/Charger):
   ✓ Dans main.ipynb, ajouter à la fin:
     import pickle
     with open('cleaned_tokens.pkl', 'wb') as f:
         pickle.dump(cleaned_tokens, f)
   ✓ Exécuter la cellule
   ✓ Revenir à main2.ipynb



## 1. Construction du Vocabulaire

Créer un vocabulaire avec une seule apparition pour chaque token unique.

In [ ]:
# Créer le vocabulaire à partir des tokens nettoyés
if data_loaded:
    try:
        vocab = list(set(cleaned_tokens))  # Créer un vocabulaire unique
        vocab_sorted = sorted(vocab)
        
        print(f"Vocabulaire créé à partir des tokens nettoyés")
        print(f"  Taille du vocabulaire: {len(vocab)} mots uniques")
        print(f"  Tokens nettoyés disponibles: {len(cleaned_tokens)}\n")
        
        print(f"Premiers 20 mots du vocabulaire: {vocab_sorted[:20]}\n")
        print(f"Derniers 20 mots du vocabulaire: {vocab_sorted[-20:]}")
        
    except NameError:
        print("⚠️  cleaned_tokens non disponible - Exécuter d'abord la cellule 0!")
        vocab = []
else:
    print("⚠️  Données non chargées - Exécuter la cellule 0 d'abord!")
    vocab = []

## 2. OneHotEncoder - Encodage One-Hot

Chaque mot unique est représenté par un vecteur où seul son index est à 1, le reste à 0.

In [ ]:
class OneHotEncoder:
    """
    Encodeur One-Hot personnalisé (style Java).
    Chaque mot unique est représenté par un vecteur de 0 et 1.
    
    Attributs:
        vocabulary: dict - Mapping {mot: index}
        vocab_size: int - Taille du vocabulaire
    """
    
    def __init__(self, vocabulary):
        """
        Initialiser l'encodeur avec un vocabulaire.
        
        Args:
            vocabulary: list - Liste des mots uniques
        """
        # Créer un dictionnaire {mot: index}
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        print(f"✓ OneHotEncoder initialisé avec {self.vocab_size} mots")
    
    def encode_word(self, word):
        """
        Encoder un seul mot en vecteur one-hot.
        
        Args:
            word: str - Le mot à encoder
        
        Returns:
            list - Vecteur one-hot de taille vocab_size
        """
        if word not in self.vocabulary:
            raise ValueError(f"Mot '{word}' non trouvé dans le vocabulaire")
        
        # Créer un vecteur de zéros
        one_hot_vector = [0] * self.vocab_size
        # Mettre 1 à l'index du mot
        one_hot_vector[self.vocabulary[word]] = 1
        
        return one_hot_vector
    
    def encode_sentence(self, words):
        """
        Encoder une phrase (liste de mots) en liste de vecteurs one-hot.
        
        Args:
            words: list - Liste de mots
        
        Returns:
            list - Liste de vecteurs one-hot
        """
        encoded = []
        for word in words:
            if word in self.vocabulary:
                encoded.append(self.encode_word(word))
        return encoded
    
    def decode_vector(self, vector):
        """
        Décoder un vecteur one-hot en mot.
        
        Args:
            vector: list - Vecteur one-hot
        
        Returns:
            str - Le mot correspondant (ou None si pas valide)
        """
        if vector.count(1) != 1:
            return None
        
        index = vector.index(1)
        # Trouver le mot correspondant à cet index
        for word, idx in self.vocabulary.items():
            if idx == index:
                return word
        return None
    
    def get_vocabulary_info(self):
        """Retourner les infos du vocabulaire"""
        return {
            'size': self.vocab_size,
            'vocabulary': self.vocabulary
        }


# Tester OneHotEncoder
print("=" * 70)
print("TEST OneHotEncoder")
print("=" * 70 + "\n")

if vocab:
    # Créer l'encodeur
    encoder = OneHotEncoder(vocab)
    
    # Tester avec un mot
    test_word = vocab_sorted[0]
    print(f"Encodage du mot '{test_word}':")
    encoded = encoder.encode_word(test_word)
    print(f"  Vecteur: {encoded}")
    print(f"  Index du 1: {encoded.index(1)}\n")
    
    # Tester avec plusieurs mots
    test_words = vocab_sorted[:5]
    print(f"Encodage des mots {test_words}:")
    for word in test_words:
        vec = encoder.encode_word(word)
        print(f"  {word:<15} → {vec.index(1):3d} (1 à la position {vec.index(1)})")
    
    # Tester le décodage
    print(f"\nDécodage des vecteurs:")
    for word in test_words:
        vec = encoder.encode_word(word)
        decoded = encoder.decode_vector(vec)
        print(f"  Vecteur → {decoded}")
    
    # Statistiques
    print(f"\n{'─' * 70}")
    print(f"Taille du vocabulaire: {encoder.vocab_size} mots")
    print(f"Dimension de chaque vecteur one-hot: {encoder.vocab_size}")
    print(f"Sparsité: ~99.85% de zéros (pour chaque vecteur)")

## 3. Bag of Words - Représentation par Fréquences

Compter l'occurrence de chaque mot dans un document (ou ensemble de mots).

In [ ]:
class BagOfWords:
    """
    Bag of Words personnalisé (style Java).
    Représente un texte par les fréquences d'apparition de chaque mot.
    
    Attributs:
        vocabulary: dict - Mapping {mot: index}
        vocab_size: int - Taille du vocabulaire
    """
    
    def __init__(self, vocabulary):
        """
        Initialiser le BoW avec un vocabulaire.
        
        Args:
            vocabulary: list - Liste des mots uniques
        """
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        print(f"✓ BagOfWords initialisé avec {self.vocab_size} mots")
    
    def encode_document(self, words):
        """
        Encoder un document (liste de mots) en vecteur de fréquences.
        
        Args:
            words: list - Liste des mots du document
        
        Returns:
            list - Vecteur de fréquences pour chaque mot du vocabulaire
        """
        # Créer un vecteur de zéros
        bow_vector = [0] * self.vocab_size
        
        # Compter les occurrences
        for word in words:
            if word in self.vocabulary:
                idx = self.vocabulary[word]
                bow_vector[idx] += 1
        
        return bow_vector
    
    def encode_documents(self, documents):
        """
        Encoder plusieurs documents.
        
        Args:
            documents: list - Liste de listes de mots
        
        Returns:
            list - Liste de vecteurs BoW
        """
        encoded_docs = []
        for doc in documents:
            encoded_docs.append(self.encode_document(doc))
        return encoded_docs
    
    def get_word_frequencies(self, vector):
        """
        Obtenir les fréquences de chaque mot à partir d'un vecteur BoW.
        
        Args:
            vector: list - Vecteur BoW
        
        Returns:
            dict - {mot: fréquence} pour les mots avec fréquence > 0
        """
        frequencies = {}
        for word, idx in self.vocabulary.items():
            if vector[idx] > 0:
                frequencies[word] = vector[idx]
        return frequencies
    
    def get_top_words(self, vector, top_n=10):
        """
        Obtenir les top N mots les plus fréquents.
        
        Args:
            vector: list - Vecteur BoW
            top_n: int - Nombre de mots à retourner
        
        Returns:
            list - Liste de tuples (mot, fréquence) triés par fréquence décroissante
        """
        frequencies = self.get_word_frequencies(vector)
        # Trier par fréquence décroissante
        sorted_freq = sorted(frequencies.items(), key=lambda x: x[1], reverse=True)
        return sorted_freq[:top_n]
    
    def get_vocabulary_info(self):
        """Retourner les infos du vocabulaire"""
        return {
            'size': self.vocab_size,
            'vocabulary': self.vocabulary
        }


# Tester BagOfWords
print("=" * 70)
print("TEST Bag of Words")
print("=" * 70 + "\n")

if vocab:
    # Créer le BoW
    bow = BagOfWords(vocab)
    
    # Utiliser les tokens nettoyés comme un seul document
    bow_vector = bow.encode_document(cleaned_tokens)
    
    print(f"Document: {len(cleaned_tokens)} mots")
    print(f"Vecteur BoW créé avec {bow.vocab_size} dimensions\n")
    
    # Obtenir les fréquences
    frequencies = bow.get_word_frequencies(bow_vector)
    print(f"Nombre de mots uniques avec fréquence > 0: {len(frequencies)}\n")
    
    # Top 15 mots
    print("Top 15 mots les plus fréquents:")
    print("─" * 40)
    top_words = bow.get_top_words(bow_vector, 15)
    for word, freq in top_words:
        pourcentage = (freq / sum(bow_vector)) * 100
        barre = "█" * int(pourcentage / 2)
        print(f"{word:<15} | {freq:3d} | {pourcentage:5.2f}% {barre}")
    
    # Statistiques
    print(f"\n{'─' * 70}")
    print(f"Vecteur BoW - Première moitié: {bow_vector[:10]}")
    print(f"Nombre total de mots: {sum(bow_vector)}")
    print(f"Moyenne d'occurrences par mot: {sum(bow_vector) / len(frequencies):.2f}")
    print(f"Sparsité: {(len([x for x in bow_vector if x == 0]) / len(bow_vector) * 100):.2f}% de zéros")

## 4. Comparaison OneHotEncoder vs Bag of Words

Voir les différences entre les deux approches.

In [ ]:
print("=" * 70)
print("COMPARAISON OneHotEncoder vs Bag of Words")
print("=" * 70 + "\n")

if vocab:
    # Sélectionner 5 mots pour la démonstration
    demo_words = vocab_sorted[10:15]
    
    print(f"Mots de démonstration: {demo_words}\n")
    
    # === OneHotEncoder ===
    print("OneHotEncoder - Représentation binaire:")
    print("─" * 70)
    print("Chaque mot a EXACTEMENT 1 seul 1, le reste 0\n")
    
    for word in demo_words:
        vec = encoder.encode_word(word)
        # Afficher le vecteur de manière compacte
        display = [1 if v == 1 else 0 for v in vec]
        print(f"{word:<15} → Position {vec.index(1):4d} → Vecteur: [1 au rang {vec.index(1)}, 0 ailleurs]")
    
    # === Bag of Words ===
    print("\n\nBag of Words - Représentation par fréquences:")
    print("─" * 70)
    print("Chaque valeur représente le nombre d'occurrences du mot\n")
    
    # Créer un petit document avec répétitions
    demo_document = demo_words + [demo_words[0], demo_words[1], demo_words[2]]
    bow_vector = bow.encode_document(demo_document)
    
    print(f"Document de test: {demo_document}")
    print(f"Longueur: {len(demo_document)} mots\n")
    
    frequencies = bow.get_word_frequencies(bow_vector)
    for word in demo_words:
        freq = frequencies.get(word, 0)
        barre = "█" * freq
        print(f"{word:<15} → Fréquence: {freq} {barre}")
    
    # === Comparaison Tableau ===
    print("\n\n" + "=" * 70)
    print("TABLEAU COMPARATIF")
    print("=" * 70)
    
    comparison = f"""
┌─────────────────────┬──────────────────────┬──────────────────────┐
│ CARACTÉRISTIQUE     │ OneHotEncoder        │ Bag of Words         │
├─────────────────────┼──────────────────────┼──────────────────────┤
│ Valeurs             │ 0 ou 1               │ Entiers (compte)     │
│ 1 par vecteur       │ Oui (exactement 1)   │ Non (variable)       │
│ Conserve l'ordre    │ Non                  │ Non                  │
│ Préserve fréquences │ Non                  │ Oui                  │
│ Sparsité            │ Très haute (~99.9%)  │ Très haute (~90%+)   │
│ Dimension           │ Taille vocabulaire   │ Taille vocabulaire   │
│ Cas d'usage         │ Classification       │ Recherche, BoW       │
│ Mem/Calcul          │ Faible               │ Léger supérieur      │
└─────────────────────┴──────────────────────┴──────────────────────┘
"""
    print(comparison)
    
    print("TAILLES MÉMOIRE COMPARÉES:")
    print("─" * 70)
    vocab_size = len(vocab)
    
    # OneHot: 1 vecteur = vocab_size bits
    onehot_size = vocab_size / 8  # bits to bytes
    
    # BoW avec dense storage: vocab_size entiers (assume 4 bytes par entier)
    bow_dense = (vocab_size * 4) / 1024  # to KB
    
    # BoW avec sparse storage: seulement les mots avec fréquence > 0
    bow_sparse = (len(frequencies) * 8) / 1024  # 2 values per entry (word + freq)
    
    print(f"Vocabulaire: {vocab_size} mots")
    print(f"OneHotEncoder par mot: ~{onehot_size:.2f} bytes")
    print(f"Bag of Words (dense): {bow_dense:.2f} KB par document")
    print(f"Bag of Words (sparse): ~{bow_sparse:.2f} KB par document")
    print(f"  → Gain mémoire (sparse): {(1 - bow_sparse/bow_dense)*100:.1f}%")

## 5. Résumé et Utilisation

Guide pratique pour utiliser les deux encodeurs.

In [ ]:
print("=" * 70)
print("RÉSUMÉ - PIPELINE COMPLET")
print("=" * 70 + "\n")

summary = """
ÉTAPE 1: VOCABULAIRE (main.ipynb)
├─ cleaned_tokens → Mots prétraités
└─ vocab = set(cleaned_tokens) → Vocabulaire unique

ÉTAPE 2: OneHotEncoder (main2.ipynb)
├─ Entrée: Vocabulaire + Mot
├─ Sortie: Vecteur [0,0,...,1,...,0] (n dimensions)
├─ Utilisation: 
│  • Classification binaire
│  • Encodage de catégories
│  • Entrée de réseaux de neurones
└─ Limite: Perd les informations de fréquence

ÉTAPE 3: Bag of Words (main2.ipynb)
├─ Entrée: Vocabulaire + Document (liste de mots)
├─ Sortie: Vecteur [f1, f2, ..., fn] (fréquences)
├─ Utilisation:
│  • Analyse de sentiment
│  • Recherche de similarité
│  • Modèles probabilistes
└─ Avantage: Conserve les fréquences

VOCABULAIRE CRÉE:
├─ Taille: """ + str(len(vocab)) + """ mots uniques
├─ Utilisé par: OneHotEncoder et BagOfWords
└─ Indexation: 0 à """ + str(len(vocab)-1) + """

DONNÉES:
├─ Tokens nettoyés: """ + str(len(cleaned_tokens)) + """ mots
├─ Tokens uniques: """ + str(len(vocab)) + """ mots
└─ Densité: """ + f"{(len(vocab)/len(cleaned_tokens)*100):.1f}%" + """
"""

print(summary)

# Exemple d'utilisation complet
print("\n" + "=" * 70)
print("EXEMPLE D'UTILISATION COMPLET")
print("=" * 70 + "\n")

if vocab:
    # Diviser en phrases pour la démo
    sentences = []
    current_sentence = []
    
    for token in cleaned_tokens[:100]:  # Utiliser les 100 premiers tokens
        current_sentence.append(token)
        if len(current_sentence) >= 8:  # Phrase de 8 mots
            sentences.append(current_sentence)
            current_sentence = []
    
    if current_sentence:
        sentences.append(current_sentence)
    
    print(f"Nombre de phrases créées: {len(sentences)}\n")
    
    # Encoder avec BoW
    print("Encoding avec Bag of Words:")
    print("─" * 70)
    
    bow_documents = bow.encode_documents(sentences[:3])  # Premières 3 phrases
    
    for i, (sent, bow_vec) in enumerate(zip(sentences[:3], bow_documents)):
        print(f"\nPhrase {i+1}: {' '.join(sent)}")
        print(f"Longueur: {len(sent)} mots")
        freq_dict = bow.get_word_frequencies(bow_vec)
        print(f"Mots uniques: {len(freq_dict)}")
        top = bow.get_top_words(bow_vec, 5)
        print(f"Top 5: {top}")

print("\n" + "=" * 70)
print("✓ Vectorisation terminée!")
print("  • Vocabulaire créé ✓")
print("  • OneHotEncoder fonctionnel ✓")
print("  • Bag of Words fonctionnel ✓")
print("=" * 70)

## 6. Modèles Word Embeddings Avancés

Implémentation de Continuous Bag of Words (CBOW), Skip-Gram, et approches distribuées.

In [ ]:
import numpy as np
from collections import defaultdict

print("=" * 70)
print("WORD EMBEDDINGS AVANCÉS")
print("=" * 70 + "\n")

# ============================================================================
# 1. CONTINUOUS BAG OF WORDS (CBOW)
# ============================================================================

class ContinuousBagOfWords:
    """
    Continuous Bag of Words (CBOW).
    Prédit un mot target à partir du contexte environnant (mots avant/après).
    
    Exemple: "Le [?] marche dans la rue"
    Contexte: [le, marche, dans] → Cible: chat
    """
    
    def __init__(self, vocabulary, window_size=2, embedding_dim=10):
        """
        Initialiser CBOW.
        
        Args:
            vocabulary: list - Vocabulaire unique
            window_size: int - Nombre de mots de contexte de chaque côté
            embedding_dim: int - Dimension des embeddings
        """
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        # Initialiser les embeddings aléatoirement
        np.random.seed(42)
        self.embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f"✓ CBOW initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_context_windows(self, words):
        """
        Extraire les fenêtres de contexte d'une liste de mots.
        
        Args:
            words: list - Séquence de mots
        
        Returns:
            list - [(contexte, target), ...]
        """
        windows = []
        for i in range(len(words)):
            # Déterminer les limites du contexte
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            # Créer le contexte (sans le mot target)
            context = []
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    context.append(words[j])
            
            # Ajouter si contexte non vide
            if context and words[i] in self.vocabulary:
                windows.append((context, words[i]))
        
        return windows
    
    def get_embedding(self, word):
        """Obtenir l'embedding d'un mot"""
        if word in self.vocabulary:
            idx = self.vocabulary[word]
            return self.embeddings[idx]
        return None
    
    def get_context_vector(self, context_words):
        """Moyenne des embeddings du contexte"""
        vectors = [self.get_embedding(w) for w in context_words if w in self.vocabulary]
        if vectors:
            return np.mean(vectors, axis=0)
        return np.zeros(self.embedding_dim)
    
    def predict_word_from_context(self, context_words, top_n=5):
        """Prédire le mot le plus probable selon le contexte"""
        context_vec = self.get_context_vector(context_words)
        
        # Calculer la similarité avec tous les mots
        similarities = []
        for word, idx in self.vocabulary.items():
            sim = np.dot(context_vec, self.embeddings[idx])
            similarities.append((word, sim))
        
        # Trier par similarité décroissante
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]


# ============================================================================
# 2. SKIP-GRAM
# ============================================================================

class SkipGram:
    """
    Skip-Gram Model.
    Prédit les mots de contexte à partir du mot target.
    
    Exemple: Chat → [le, marche, dans, ...]
    Inverse de CBOW.
    """
    
    def __init__(self, vocabulary, window_size=2, embedding_dim=10):
        """
        Initialiser Skip-Gram.
        
        Args:
            vocabulary: list - Vocabulaire unique
            window_size: int - Nombre de mots de contexte de chaque côté
            embedding_dim: int - Dimension des embeddings
        """
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        # Initialiser les embeddings
        np.random.seed(42)
        self.input_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        self.output_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f"✓ Skip-Gram initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_training_pairs(self, words):
        """
        Extraire les paires (target, context) pour l'entraînement.
        
        Args:
            words: list - Séquence de mots
        
        Returns:
            list - [(target, contexte), ...]
        """
        pairs = []
        for i in range(len(words)):
            if words[i] not in self.vocabulary:
                continue
            
            target = words[i]
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    pairs.append((target, words[j]))
        
        return pairs
    
    def get_embedding(self, word):
        """Obtenir l'embedding d'un mot (input)"""
        if word in self.vocabulary:
            idx = self.vocabulary[word]
            return self.input_embeddings[idx]
        return None
    
    def predict_context(self, target_word, top_n=5):
        """Prédire les mots de contexte probables"""
        if target_word not in self.vocabulary:
            return []
        
        target_idx = self.vocabulary[target_word]
        target_vec = self.input_embeddings[target_idx]
        
        # Calculer les scores avec les embeddings de sortie
        similarities = []
        for word, idx in self.vocabulary.items():
            sim = np.dot(target_vec, self.output_embeddings[idx])
            similarities.append((word, sim))
        
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_n]


# ============================================================================
# 3. DISTRIBUTED BAG OF WORDS (DBOW)
# ============================================================================

class DistributedBagOfWords:
    """
    Distributed Bag of Words (Doc2Vec - PV-DBOW).
    Représente chaque document par un vecteur continu.
    """
    
    def __init__(self, vocabulary, embedding_dim=20):
        """
        Initialiser Distributed BoW.
        
        Args:
            vocabulary: list - Vocabulaire unique
            embedding_dim: int - Dimension des embeddings
        """
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.embedding_dim = embedding_dim
        
        # Embeddings des mots et des documents
        np.random.seed(42)
        self.word_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f"✓ Distributed Bag of Words initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}\n")
    
    def get_document_vector(self, words, doc_id=None):
        """
        Créer un vecteur pour un document.
        Moyenne des embeddings des mots + ID du document.
        
        Args:
            words: list - Mots du document
            doc_id: int - Identifiant unique du document
        
        Returns:
            np.array - Vecteur du document
        """
        # Moyenner les embeddings des mots
        word_vecs = [self.word_embeddings[self.vocabulary[w]] 
                     for w in words if w in self.vocabulary]
        
        if word_vecs:
            doc_vec = np.mean(word_vecs, axis=0)
        else:
            doc_vec = np.zeros(self.embedding_dim)
        
        return doc_vec
    
    def get_document_similarity(self, doc1_words, doc2_words):
        """Calculer la similarité cosinus entre deux documents"""
        vec1 = self.get_document_vector(doc1_words)
        vec2 = self.get_document_vector(doc2_words)
        
        norm1 = np.linalg.norm(vec1)
        norm2 = np.linalg.norm(vec2)
        
        if norm1 > 0 and norm2 > 0:
            return np.dot(vec1, vec2) / (norm1 * norm2)
        return 0.0


# ============================================================================
# 4. DISTRIBUTED MEMORY (DM)
# ============================================================================

class DistributedMemory:
    """
    Distributed Memory (Doc2Vec - PV-DM).
    Combine le contexte des mots ET l'ID du document pour prédire.
    """
    
    def __init__(self, vocabulary, window_size=2, embedding_dim=20):
        """
        Initialiser Distributed Memory.
        
        Args:
            vocabulary: list - Vocabulaire unique
            window_size: int - Taille du contexte
            embedding_dim: int - Dimension des embeddings
        """
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(vocabulary))}
        self.vocab_size = len(self.vocabulary)
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        # Embeddings
        np.random.seed(42)
        self.word_embeddings = np.random.randn(self.vocab_size, embedding_dim)
        
        print(f"✓ Distributed Memory initialisé")
        print(f"  Vocabulaire: {self.vocab_size} mots")
        print(f"  Dimension des embeddings: {embedding_dim}")
        print(f"  Taille du contexte: ±{window_size} mots\n")
    
    def get_context_windows_with_doc(self, words, doc_id=0):
        """
        Extraire les fenêtres incluant l'ID du document.
        
        Args:
            words: list - Mots du document
            doc_id: int - Identifiant du document
        
        Returns:
            list - [(contexte + doc_id, target), ...]
        """
        windows = []
        for i in range(len(words)):
            if words[i] not in self.vocabulary:
                continue
            
            start = max(0, i - self.window_size)
            end = min(len(words), i + self.window_size + 1)
            
            context = []
            for j in range(start, end):
                if j != i and words[j] in self.vocabulary:
                    context.append(words[j])
            
            if context:
                # Ajouter l'ID du document au contexte
                context_with_doc = context + [f"DOC_{doc_id}"]
                windows.append((context_with_doc, words[i]))
        
        return windows
    
    def get_document_representation(self, words, doc_id=0):
        """
        Obtenir une représentation du document combinant
        les mots et l'ID du document.
        """
        word_vecs = [self.word_embeddings[self.vocabulary[w]] 
                     for w in words if w in self.vocabulary]
        
        if word_vecs:
            return np.mean(word_vecs, axis=0)
        return np.zeros(self.embedding_dim)


# ============================================================================
# TESTS ET COMPARAISONS
# ============================================================================

print("=" * 70)
print("TESTS DES MODÈLES")
print("=" * 70 + "\n")

if vocab and data_loaded:
    # Créer les modèles
    cbow = ContinuousBagOfWords(vocab, window_size=2, embedding_dim=10)
    skipgram = SkipGram(vocab, window_size=2, embedding_dim=10)
    dbow = DistributedBagOfWords(vocab, embedding_dim=20)
    dm = DistributedMemory(vocab, window_size=2, embedding_dim=20)
    
    # Utiliser les premiers tokens
    sample_words = cleaned_tokens[:50]
    
    # === Test CBOW ===
    print("─" * 70)
    print("1. CONTINUOUS BAG OF WORDS (CBOW)")
    print("─" * 70)
    print("Prédit un mot à partir du contexte environnant\n")
    
    cbow_windows = cbow.get_context_windows(sample_words)
    print(f"Fenêtres de contexte extraites: {len(cbow_windows)}\n")
    
    # Exemple
    if cbow_windows:
        context, target = cbow_windows[0]
        print(f"Exemple:")
        print(f"  Contexte: {context}")
        print(f"  Cible réelle: {target}")
        predictions = cbow.predict_word_from_context(context, top_n=5)
        print(f"  Top 5 prédictions:")
        for word, score in predictions:
            print(f"    {word:<15} score: {score:.4f}")
    
    # === Test Skip-Gram ===
    print("\n" + "─" * 70)
    print("2. SKIP-GRAM")
    print("─" * 70)
    print("Prédit le contexte à partir du mot cible\n")
    
    skipgram_pairs = skipgram.get_training_pairs(sample_words)
    print(f"Paires (target, contexte) créées: {len(skipgram_pairs)}\n")
    
    # Exemple
    if skipgram_pairs:
        target, context = skipgram_pairs[0]
        print(f"Exemple:")
        print(f"  Mot cible: {target}")
        print(f"  Contexte réel: {context}")
        predictions = skipgram.predict_context(target, top_n=5)
        print(f"  Top 5 contextes prédits:")
        for word, score in predictions:
            print(f"    {word:<15} score: {score:.4f}")
    
    # === Test Distributed Bag of Words ===
    print("\n" + "─" * 70)
    print("3. DISTRIBUTED BAG OF WORDS (DBOW)")
    print("─" * 70)
    print("Représente un document par un vecteur continu\n")
    
    # Diviser en documents
    doc1 = cleaned_tokens[:30]
    doc2 = cleaned_tokens[30:60]
    doc3 = cleaned_tokens[60:90]
    
    vec1 = dbow.get_document_vector(doc1)
    vec2 = dbow.get_document_vector(doc2)
    vec3 = dbow.get_document_vector(doc3)
    
    print(f"Vecteur document 1: {vec1[:5]}... (taille: {vec1.shape})")
    print(f"Vecteur document 2: {vec2[:5]}... (taille: {vec2.shape})")
    print(f"Vecteur document 3: {vec3[:5]}... (taille: {vec3.shape})\n")
    
    # Similarité entre documents
    sim_1_2 = dbow.get_document_similarity(doc1, doc2)
    sim_1_3 = dbow.get_document_similarity(doc1, doc3)
    sim_2_3 = dbow.get_document_similarity(doc2, doc3)
    
    print("Similarités cosinus entre documents:")
    print(f"  Doc1 vs Doc2: {sim_1_2:.4f}")
    print(f"  Doc1 vs Doc3: {sim_1_3:.4f}")
    print(f"  Doc2 vs Doc3: {sim_2_3:.4f}")
    
    # === Test Distributed Memory ===
    print("\n" + "─" * 70)
    print("4. DISTRIBUTED MEMORY (DM)")
    print("─" * 70)
    print("Combine contexte ET ID du document pour prédire\n")
    
    dm_windows = dm.get_context_windows_with_doc(sample_words[:30], doc_id=1)
    print(f"Fenêtres (contexte + doc_id, target) créées: {len(dm_windows)}\n")
    
    if dm_windows:
        context, target = dm_windows[0]
        print(f"Exemple:")
        print(f"  Contexte + DocID: {context}")
        print(f"  Cible: {target}")
    
    doc_repr = dm.get_document_representation(sample_words[:30], doc_id=1)
    print(f"  Représentation du doc: {doc_repr[:5]}... (taille: {doc_repr.shape})")

print("\n" + "=" * 70)
print("✓ Modèles Word Embeddings terminés!")
print("=" * 70)

## 7. Comparaison des Modèles

Tableau récapitulatif des différentes approches.

In [ ]:
print("=" * 80)
print("COMPARAISON COMPLÈTE DES MODÈLES DE VECTORISATION")
print("=" * 80 + "\n")

comparison_table = """
┌─────────────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ MODÈLE              │ OneHot/BoW   │ CBOW         │ Skip-Gram    │ DBOW/DM      │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Type                │ Comptage     │ Prédiction   │ Prédiction   │ Embedding    │
│ Entrée              │ Mots         │ Contexte     │ Mot          │ Mots+Doc     │
│ Sortie              │ Vecteur 0/1  │ Mot prédit   │ Contexte     │ Embedding    │
│ Apprentissage       │ Non          │ Neuronal     │ Neuronal     │ Neuronal     │
│ Dimension           │ |V| (sparse) │ embedding    │ embedding    │ embedding    │
│ Sémantique          │ Non          │ Oui          │ Oui          │ Oui          │
│ Similarité          │ Jaccard      │ Cosinus      │ Cosinus      │ Cosinus      │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Avantages           │ • Simple     │ • Rapide     │ • Flexible   │ • Document   │
│                     │ • Sparse     │ • Efficace   │ • Détaillé   │ • Contexte   │
│                     │ • Rapide     │ • Efficace   │ • Hiérarchie │ • Flexible   │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Inconvénients       │ • Pas séman. │ • Plus lent  │ • Plus lent  │ • Complexe   │
│                     │ • Perte info │ • GPU besoin │ • GPU besoin │ • Training   │
│                     │ • Haute dim. │              │              │              │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Cas d'usage         │ Recherche    │ Prédiction   │ Similarité   │ Classification│
│                     │ Fil. contenu │ de contexte  │ de contexte  │ de docs      │
│                     │ TF-IDF       │ Embedding    │ Embedding    │ Clustering   │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Complexité          │ O(|V|)       │ O(C×D)       │ O(C×D)       │ O(D×|V|)     │
│                     │ Bas          │ Moyen        │ Moyen        │ Moyen        │
├─────────────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│ Équivalent          │ TF-IDF       │ Word2Vec     │ Word2Vec     │ Doc2Vec      │
│ Réel                │ fastText     │ GloVe        │ FastText     │ Doc2Vec      │
└─────────────────────┴──────────────┴──────────────┴──────────────┴──────────────┘

LÉGENDE:
  |V|     = Taille du vocabulaire
  C       = Taille du contexte
  D       = Dimension des embeddings
  O()     = Complexité temporelle
"""

print(comparison_table)

# Visualisation simple
print("\n" + "=" * 80)
print("REPRÉSENTATION SCHÉMATIQUE")
print("=" * 80 + "\n")

schemas = """
1. CONTINUOUS BAG OF WORDS (CBOW)
   ┌─────────┐  ┌─────────┐  ┌─────────┐
   │  "Le"   │  │ "chat"  │  │  "dort" │
   └────┬────┘  └────┬────┘  └────┬────┘
        │            │            │
        └─────────────┼─────────────┘
                      ↓
              [Réseau de Neurones]
                      ↓
              Prédire: "marche"
              
2. SKIP-GRAM
   ┌─────────┐
   │ "chat"  │ ← Mot cible
   └────┬────┘
        ↓
   [Réseau de Neurones]
        ↓
   ┌────┴─────┬──────┬──────┐
   ↓          ↓      ↓      ↓
  "le"      "dort" "dans" "rue"
  (contexte prédit)
  
3. DISTRIBUTED BAG OF WORDS (DBOW)
   Document_ID: [1]
   Mots: [chat, marche, rue]
        ┌─────┬──────┬────┐
        ↓     ↓      ↓    ↓
   [Réseau avec Document_ID]
        ↓
   Vecteur du document: [0.2, -0.1, 0.5, ...]
   
4. DISTRIBUTED MEMORY (DM)
   Document_ID + Contexte: [DOC_1, "le", "chat"]
        ┌──────┬────┬────┐
        ↓      ↓    ↓    ↓
   [Réseau de Neurones]
        ↓
   Prédire mot suivant + apprendre vecteur du doc
"""

print(schemas)

# Résumé statistique
print("\n" + "=" * 80)
print("RÉSUMÉ STATISTIQUE")
print("=" * 80 + "\n")

if vocab and data_loaded:
    summary_stats = f"""
DONNÉES DISPONIBLES:
  • Vocabulaire: {len(vocab)} mots uniques
  • Tokens nettoyés: {len(cleaned_tokens)} mots
  • Densité: {(len(vocab)/len(cleaned_tokens)*100):.2f}%

MODÈLES IMPLÉMENTÉS:
  1. OneHot Encoder
     - Dimensions: {len(vocab)}
     - Sparsité: ~99.9%
     
  2. Bag of Words
     - Dimensions: {len(vocab)}
     - Représentation: fréquences
     
  3. Continuous Bag of Words (CBOW)
     - Contexte: ±2 mots
     - Embedding: 10 dimensions
     - Fenêtres: ~{len(cbow_windows)} paires
     
  4. Skip-Gram
     - Contexte: ±2 mots
     - Embedding: 10 dimensions
     - Paires training: ~{len(skipgram_pairs)} pairs
     
  5. Distributed Bag of Words (DBOW)
     - Embedding: 20 dimensions
     - Docs testés: 3
     
  6. Distributed Memory (DM)
     - Contexte: ±2 mots
     - Embedding: 20 dimensions
     - Avec Doc ID: Oui

PROCHAINES ÉTAPES:
  ✓ Visualiser les embeddings (t-SNE, PCA)
  ✓ Entraîner les modèles avec gradient descent
  ✓ Évaluer la qualité des embeddings
  ✓ Comparaison avec Word2Vec réel
"""
    print(summary_stats)

print("\n" + "=" * 80)
print("✓ Section Word Embeddings Avancés terminée!")
print("=" * 80)